## Libraries

In [1]:
import pandas as pd #dataframes e series
import matplotlib.pyplot as plt #graphics
import numpy as np #matrixes
import csv #read files
import nltk #Tokenize
import re #RegEx
import string 
import collections
import scipy #Sparse Matrixes

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer #tf-idf; tf
from sklearn.naive_bayes import MultinomialNB #Naïve Bayes
from sklearn import metrics #Evaluation
from sklearn.linear_model import LogisticRegression
from nltk.tokenize import word_tokenize
from sklearn import svm
from sklearn.feature_selection import SelectFromModel
import multiprocessing
from sklearn.ensemble import ExtraTreesClassifier

In [3]:
from ipywidgets import FloatProgress
from IPython.display import display

In [4]:
plt.rcParams['figure.figsize'] = (20,10)
plt.style.use('ggplot')

In [5]:
Trainingtable = pd.read_csv('C:/Users/sergiojesus/Desktop/Recursos/comments_toxicos/Dados/train.csv')
Testtable = pd.read_csv('C:/Users/sergiojesus/Desktop/Recursos/comments_toxicos/Dados/test.csv')

In [6]:
label_cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [9]:
Trainingcomments = Trainingtable.comment_text
Testcomments = Testtable.comment_text

In [ ]:
num_rows = Trainingtable.shape[0] #Número de linhas
num_missing = num_rows - Trainingtable.count()

In [ ]:
print(Trainingtable.comment_text.value_counts(dropna=False)) # Print da ocorrência de valores numa dada coluna

In [ ]:
Trainingtable.describe()

In [ ]:
subset = Trainingtable.iloc[[42, 44], [3]]
subset

In [ ]:
Trainingtable.groupby('toxic').mean()

   Para qualquer feature, P(toxic|feature = true) = 93%, excepto P(toxic|severe_toxic = true) = 100 %  
   P(toxic)= 9,5844%  


In [ ]:
Trainingtable['f_sum']= (summ) #adicionar nova coluna à tabela de treino

In [ ]:
plt.hist(Trainingtable['sum'], bins = 6)
plt.show() #histograma da soma de características negativas

In [ ]:
Trainingtable['sum'].describe()

In [ ]:
summ6 = summ[summ == 6]
for i in summ6.index:
    print(Trainingtable['comment_text'][i])  #Print dos comentários que infringem todas as  para ver padrões

In [ ]:
comments = Trainingtable['comment_text'] #Criação de uma pandas.Series dos comentários
uppercasecount = pd.Series() #Criação do contador em forma de pandas.Series
for i in range(comments.shape[0]): 
    comment = comments[i]
    count = 0
    for c in comment:
        if c.isupper():
            count += 1 #Programado intuitivamente, não tem compreensão de listas
    uppercasecount = uppercasecount.append(pd.Series([count], index = [i])) 

In [ ]:
Trainingtable['uppercase_count'] = uppercasecount #Inserir o número de maiúsculas por comentário na tabela de treino

In [ ]:
Trainingtable[['sum','uppercase_count']].groupby('sum').mean()

In [ ]:
with open('C:/Users/sergiojesus/Desktop/Recursos/comments_toxicos/full-list-of-bad-words-banned-by-google.csv') as f:
  reader = csv.reader(f)
  swearsBoW = list(reader)
for i in range(1,len(swearsBoW)):
    for word in swearsBoW[i]:
            swearsBoW[0].append(word)
swearsBoW = swearsBoW[0]
newlist= list()
for word in swearsBoW:
    newword = word.strip()
    newword2 = newword.lower()
    newlist.append(newword2)
swearsBoW = newlist

In [ ]:
swear_count = pd.Series()
wordlist = [comment.split() for comment in comments]
commentcount = 0
for sentence in wordlist:
    count = 0
    for word in sentence:
        lowerword = word.lower()
        if lowerword in swearsBoW:
            count += 1
    swear_count = swear_count.append(pd.Series([count], index = [commentcount]))
    commentcount += 1
    f.value = commentcount

In [ ]:
Trainingtable['swear_count'] = swear_count

In [ ]:
Trainingtable[['sum','swear_count']].groupby('sum').mean()

In [ ]:
Trainingtable['uppercase_count'].describe()

In [ ]:
# Tentativa de criação de um Bag of Words para os comentários todos
#Sem compreensão de listas por enquanto
BagofWords = dict() 
commentcount = 0
for comment in wordlist:
    for word in comment:
        BagofWords[word.lower()] = 0
    commentcount += 1
    f.value = commentcount/2
for comment in wordlist:
    for word in comment:
        BagofWords[word.lower()] += 1
    commentcount += 1
    f.value = commentcount/2

In [ ]:
BagofWords #ainda tem pontuação, palavras com 1 char, etc.

In [ ]:
with open('C:/Users/sergiojesus/Desktop/Recursos/comments_toxicos/full-list-of-bad-words-banned-by-google.csv') as f:
  reader = csv.reader(f)
  swearsBoW1 = list(reader)

In [ ]:
swearsBoW1 = [nltk.word_tokenize(a) for a in swearsBoW]

In [ ]:
CommentTokens = [nltk.word_tokenize(a) for a in comments]
CommentTokens = [item for sublist in CommentTokens for item in sublist] #itertools.chain para o mesmo resultado

In [ ]:
swearsTokens = [a for b in swearsBoW1 for a in b]

In [ ]:
lens = Trainingtable.comment_text.str.len()

In [ ]:
plt.hist(lens)
plt.show()

In [ ]:
swear = list()
for i in Trainingtable.swear_count:
    if i > 0:
        swear.append(1)
    else:
        swear.append(0)

In [ ]:
Trainingtable['swear'] = swear

In [ ]:
Trainingtable

In [ ]:
string.punctuation

In [ ]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
print(type(re_tok))

In [ ]:
def tokenize(s): return re_tok.sub(r' \1 ', s).split()
tokenize(Trainingtable['comment_text'])

## *Naïve Bayes* 
##### (com imput = freq de token)

In [7]:
Countmodel = CountVectorizer(max_df= 0.8 , min_df= 2, tokenizer = nltk.word_tokenize, ngram_range= (1,3))
Counttraining = Countmodel.fit_transform(Trainingtable['comment_text'])
CountTest = Countmodel.transform(Testtable['comment_text'])

Segundo a página da library SKLEARN, o MNB é usado com *features* discretas, mas teoricamente pode-se usar variáveis continuas.

O melhor resultado foi obtido com o imput de frequência de tokens e com tri-grams e com P(class == True), ou seja com o método predict_proba

In [ ]:
classifier = MultinomialNB()
Prediction = pd.DataFrame()
preds = np.zeros((len(Testtable), len(label_cols)))
for i,j in enumerate(label_cols):
    classifier.fit(Counttraining, Trainingtable[j])
    preds[:,i] = classifier.predict_proba(CountTest)[:,1]

In [ ]:
Prediction = pd.DataFrame(Testtable['id'])
Prediction = pd.concat([Prediction, pd.DataFrame(preds, columns = label_cols)], axis= 1)

In [ ]:
Prediction.head()

In [ ]:
Prediction.to_csv('submission.csv', index = False)

## *Suport Vector Machines*
##### (com imput = TF-IDF)

In [33]:
Tfmodel = TfidfVectorizer(max_df= 0.8 , min_df= 2, tokenizer = nltk.word_tokenize, ngram_range= (1,3), max_features = 100000)

In [35]:
Tfidftraining = Tfmodel.fit_transform(Trainingtable['comment_text'])

In [36]:
Tfidftest = Tfmodel.transform(Testtable['comment_text'])

In [ ]:
classifierSVM = svm.SVC()
predsSVM = np.zeros((len(Testtable), len(label_cols)))
for i,j in enumerate(label_cols):
    print(j)
    classifierSVM.fit(Tfidftraining, Trainingtable[j])
    predsSVM[:,i] = classifier.predict_proba(Tfidftest)[:,1]

In [ ]:
classifierSVM = svm.SVC()

## *Logistic Regression* 
##### (com imput = freq de token) (AUC em Crossfold Validation mais alto que TF-IDF)

In [27]:
logisticmodel = LogisticRegression(C=4, dual=True)

In [28]:
Prediction = pd.DataFrame()
preds = np.zeros((len(Testtable), len(label_cols)))
for i,j in enumerate(label_cols):
    logisticmodel.fit(Counttraining, Trainingtable[j])
    preds[:,i] = logisticmodel.predict_proba(CountTest)[:,1]

C:\Users\sergiojesus\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


In [29]:
Prediction = pd.DataFrame(Testtable['id'])
Prediction = pd.concat([Prediction, pd.DataFrame(preds, columns = label_cols)], axis= 1)

In [30]:
Prediction.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,1.000000,0.000297,1.000000,0.000387,0.999949,3.586194e-01
1,0000247867823ef7,0.000467,0.000295,0.000635,0.000006,0.001265,5.611316e-04
2,00013b17ad220c46,0.006838,0.000523,0.002328,0.000018,0.000531,1.556571e-04
3,00017563c3f7919a,0.000002,0.000004,0.000003,0.000002,0.000016,4.572448e-07
4,00017695ad8997eb,0.009227,0.000882,0.002763,0.000310,0.003582,3.118968e-04


In [31]:
Prediction.to_csv('submission.csv', index = False)

In [37]:
Prediction = pd.DataFrame()
preds = np.zeros((len(Testtable), len(label_cols)))
for i,j in enumerate(label_cols):
    logisticmodel.fit(Tfidftraining, Trainingtable[j])
    preds[:,i] = logisticmodel.predict_proba(Tfidftest)[:,1]

In [38]:
Prediction = pd.DataFrame(Testtable['id'])
Prediction = pd.concat([Prediction, pd.DataFrame(preds, columns = label_cols)], axis= 1)

In [39]:
Prediction.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.999333,0.061647,0.997785,0.038995,0.914696,0.342033
1,0000247867823ef7,0.005680,0.002760,0.003503,0.000262,0.003531,0.002257
2,00013b17ad220c46,0.016632,0.003045,0.007514,0.000185,0.001380,0.000782
3,00017563c3f7919a,0.002620,0.001808,0.001613,0.000910,0.002312,0.000665
4,00017695ad8997eb,0.016137,0.001859,0.005803,0.001112,0.005920,0.000923


In [40]:
Prediction.to_csv('submission.csv', index = False)

## *NB+Logistic regression*
##### (input = TF-IDF)
Feature selection

In [8]:
Tfmodel = TfidfVectorizer(max_df= 0.8 , min_df= 2, tokenizer = nltk.word_tokenize, ngram_range= (1,3))

In [10]:
TrainingTFID = Tfmodel.fit_transform(Trainingcomments)

In [26]:
TestTFID = Tfmodel.transform(Testcomments)

In [13]:
NBclf = MultinomialNB()
NBclf.fit(TrainingTFID,Trainingtable.toxic)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
FeatSelectModel = SelectFromModel(NBclf, prefit= True)

In [20]:
NewX = FeatSelectModel.transform(TrainingTFID)

In [25]:
print(TrainingTFID.shape)
print(NewX.shape)

(159571, 1686832)
(159571, 1353252)


In [45]:
NBclf = MultinomialNB()
LogRes = LogisticRegression()
preds = np.zeros((len(Testtable), len(label_cols)))
for i,colname in enumerate(label_cols):
    NBclf.fit(TrainingTFID,Trainingtable[colname])
    FeatSelectModel = SelectFromModel(NBclf, prefit= True)
    NewX = FeatSelectModel.transform(TrainingTFID)
    NewTestX = FeatSelectModel.transform(TestTFID)
    NBclf.fit(NewX,Trainingtable[colname])
    LogResX = TestTFID.multiply(NBclf.predict_proba(NewTestX)[:,1].reshape(-1, 1))
    LogEntryX = TrainingTFID.multiply(NBclf.predict_proba(NewX)[:,1].reshape(-1, 1))
    LogRes.fit(LogEntryX,Trainingtable[colname])
    preds[:,i] = LogRes.predict_proba(LogResX)[:,1]
    print(colname)

toxic
severe_toxic
obscene
threat
insult
identity_hate


In [46]:
Prediction = pd.DataFrame(Testtable['id'])
Prediction = pd.concat([Prediction, pd.DataFrame(preds, columns = label_cols)], axis= 1)

In [47]:
Prediction.to_csv('submission.csv', index = False)

In [ ]:
def toptokens(Tfidfmatrix, tokens, n=5):
    npmatrix = Tfidfmatrix.toarray()
    npmatrix[npmatrix == 0] = np.nan
    npmeans = np.nanmean(npmatrix, axis = 0)
    topids = np.argsort(npmeans)[::-1][:n]
    toptoken = [(tokens[i], npmeans[i]) for i in topids]
    df = pd.DataFrame(toptoken)
    df.columns = ['feature', 'TF-IDF']
    return df

In [ ]:
toptokens(Tfidftraining5, Features, n = 20)

In [ ]:
print(Tfidftraining5.toarray().shape[0])

In [ ]:
summ1 = Trainingtable.toxic
summ2 = Trainingtable.severe_toxic
 = list()
for i in range(summ2.shape[0]):
    summ4.append(str(summ2[i]) +','+ str(summ3[i]))

In [ ]:
len(summ2)

In [ ]:
type(summ2)

In [ ]:
summ4

In [ ]:
def join_class(row):
    toxic = row['toxic']
    severe_toxic = row['severe_toxic']
    obscene = row['obscene']
    threat = row['threat']
    insult = row['insult']
    identity_hate = row['identity_hate']
    jclass = str(toxic) + str(severe_toxic) + str(obscene) + str(threat) + str(insult) + str(identity_hate)
    return jclass

In [ ]:
Trainingtable=Trainingtable.assign(Classes_str = Trainingtable.apply(join_class, axis = 1))

In [ ]:
%matplotlib inline
Trainingtable.groupby('Classes_str').agg('count')

In [ ]:
Trainingtable = Trainingtable.assign(comment_value = Trainingtable.apply(average_score, axis = 1))

In [ ]:
def average_score(row):
    return Tfidftraining[row.name,:].mean()

In [ ]:
print(Tfidftraining[5,:].toarray())

In [ ]:
x = Trainingtable.iloc[65738]

In [ ]:
Trainingtable['comment_value'].describe()

In [ ]:
sortedTT = Trainingtable.sort_values('comment_value', ascending= False)

In [ ]:
sortedTT.index

In [ ]:
list_classes = pd.unique(Trainingtable.Classes_str)

In [ ]:
list_classes

In [ ]:
numelement = 0
commentid_list = list()
for element in list_classes:
    commentid_list_sec = list()
    commentid_list_sec.append(sortedTT[sortedTT['Classes_str'] == element].index.values)
    numelement +=1
    commentid_list.append(commentid_list_sec)

In [ ]:
len(commentid_list)

In [ ]:
sortedTT[sortedTT['Classes_str'] == '000000']

In [ ]:
c = 0 
for i in commentid_list:
    for j in i:
        (j[:20])
c

In [ ]:
help(Int64Index)

In [ ]:
commentid_list

In [ ]:
for row in Prediction:
    print(row) 

In [ ]:
Prediction2 = pd.DataFrame(0, index = np.arange(len(Prediction)), columns = label_cols3)
Prediction2['id'] = Prediction['id']
Prediction2

In [ ]:
label_cols3 = ['id','toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [ ]:
print(Tfidftraining)

In [ ]:
def CBTW1(ntf, A, B, C, D):
    return ntf * log(1+(A/B)*(A/C))

In [ ]:
Randomcomment = Trainingtable.iloc[2,1]
print(type(Randomcomment))

In [ ]:
print(word_tokenize(Randomcomment))
Randomcomment

In [ ]:
x = ntf_matrix(Tftraining)
x

In [ ]:
from ipywidgets import FloatProgress
from IPython.display import display
f = FloatProgress(min=0, max=621305)
display(f)

In [ ]:
Tf = TfidfVectorizer(max_df= 0.8 , min_df= 2, tokenizer = nltk.word_tokenize)

In [ ]:
ntftraining = Tf.fit_transform(Trainingtable['comment_text'])

In [ ]:
row = Tftraining.getrow(3)

In [ ]:
print(row)

In [ ]:
Tftraining

In [ ]:
print(Tftraining.max())

In [ ]:
COMMENT = 'comment_text'
Trainingtable[COMMENT].fillna("unknown", inplace=True)
Testtable[COMMENT].fillna("unknown", inplace=True)

In [ ]:
print(classifier.predict_proba(CountTest))

In [ ]:
plt.spy(Counttraining)
plt.show()

In [ ]:
Counttraining.nnz

In [ ]:
Counttraining.shape[0]*Counttraining.shape[1]

In [ ]:
Counttraining.shape

In [ ]:
24413307/269169469072

In [ ]:
scipy.linalg.expm_cond(Counttraining)

## *CBTW(1)*

In [11]:
def maxvec(matrix):
    vec = list()
    for i in range(matrix.shape[0]):
        vec.append(matrix.getrow(i).max())
        f.value = i
    return vec

In [12]:
valuevec = maxvec(Counttraining)

In [10]:
f = FloatProgress(min=0, max=159571)
display(f)

A Jupyter Widget

In [21]:
ntftraining = scipy.sparse.csr_matrix((Counttraining.shape[0],Counttraining.shape[1]))
f.value = 0
for i in range(Counttraining.shape[0]):
    f.value = i
    ntftraining[i,:] = Counttraining.getrow(i) / valuevec[i]

C:\Users\sergiojesus\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\sparse\compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)
C:\Users\sergiojesus\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\sparse\base.py:470: RuntimeWarning: divide by zero encountered in true_divide
  return self.astype(np.float_)._mul_scalar(1./other)


In [14]:
np.linalg.cond(Counttraining)

LinAlgError: 0-dimensional array given. Array must be at least two-dimensional

In [15]:
Counttraining

<159571x1686832 sparse matrix of type '<class 'numpy.int64'>'
	with 24413307 stored elements in Compressed Sparse Row format>

In [23]:
print(ntftraining)

  (0, 33877)	0.333333333333
  (0, 35489)	0.333333333333
  (0, 68629)	0.333333333333
  (0, 87454)	0.333333333333
  (0, 116258)	0.333333333333
  (0, 116439)	0.333333333333
  (0, 183272)	0.333333333333
  (0, 188069)	0.333333333333
  (0, 188093)	0.333333333333
  (0, 257126)	0.333333333333
  (0, 257577)	0.333333333333
  (0, 257630)	0.333333333333
  (0, 289184)	0.333333333333
  (0, 308051)	0.333333333333
  (0, 308066)	0.333333333333
  (0, 370380)	0.333333333333
  (0, 372366)	0.333333333333
  (0, 372367)	0.333333333333
  (0, 486636)	0.333333333333
  (0, 486652)	0.333333333333
  (0, 564807)	0.333333333333
  (0, 566162)	0.333333333333
  (0, 566548)	0.333333333333
  (0, 571960)	0.333333333333
  (0, 589064)	0.333333333333
  :	:
  (159570, 1197901)	0.333333333333
  (159570, 1198394)	0.333333333333
  (159570, 1198410)	0.333333333333
  (159570, 1230677)	0.333333333333
  (159570, 1232203)	0.333333333333
  (159570, 1232572)	0.333333333333
  (159570, 1232574)	0.333333333333
  (159570, 1440515)	0.333333

In [25]:
print(Counttraining)

  (0, 761887)	1
  (0, 1282139)	1
  (0, 1120020)	1
  (0, 1349495)	1
  (0, 1428126)	1
  (0, 671735)	1
  (0, 1354440)	1
  (0, 1428468)	1
  (0, 1215613)	1
  (0, 987726)	1
  (0, 566548)	1
  (0, 1147651)	1
  (0, 308066)	1
  (0, 116439)	1
  (0, 623575)	1
  (0, 372367)	1
  (0, 257630)	1
  (0, 989006)	1
  (0, 1452182)	1
  (0, 188093)	1
  (0, 1228363)	1
  (0, 1589659)	1
  (0, 711037)	1
  (0, 1528802)	1
  (0, 589849)	1
  :	:
  (159570, 138178)	1
  (159570, 140931)	1
  (159570, 689690)	1
  (159570, 1568160)	1
  (159570, 1591003)	1
  (159570, 1659974)	1
  (159570, 123392)	1
  (159570, 8823)	1
  (159570, 191534)	1
  (159570, 716556)	1
  (159570, 1651925)	2
  (159570, 1454267)	1
  (159570, 8739)	3
  (159570, 191523)	3
  (159570, 815758)	1
  (159570, 1037557)	2
  (159570, 815598)	1
  (159570, 1197901)	1
  (159570, 566162)	1
  (159570, 564807)	1
  (159570, 289184)	2
  (159570, 760943)	2
  (159570, 68629)	1
  (159570, 982937)	1
  (159570, 975433)	1
